In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib2
import json

In [ ]:
def getBestSellerCategories():
    bsp = urllib2.urlopen("http://www.amazon.in/gp/bestsellers/").read();
    soup = BeautifulSoup(bsp,'html.parser')
    browseRoot = soup.find(id="zg_browseRoot")
    catAHref = browseRoot.find('ul').find_all('a')
    bs_categories = []
    for ch in catAHref:
        bs_categories.append((ch.text,ch.attrs['href']))
    bsc = pd.DataFrame(columns=['Name','BestSellerLink'])
    bsc.Name = [b[0] for b in bs_categories]
    bsc.BestSellerLink = [b[1] for b in bs_categories]
    return bsc
    

In [ ]:
bsc = getBestSellerCategories()

In [ ]:
bsc.to_csv('output/bestseller_links.csv',index=False)
bsc.to_json('output/bestseller_links.json',orient='records')

In [ ]:
def getCategoryBestSellers(link):
    items = []
    page_count = 1
    cur_page = 1
    while cur_page <= page_count:
        html = urllib2.urlopen(link).read()
        soup = BeautifulSoup(html,'html.parser')
        page_count = len(soup.find(id='zg_paginationWrapper').select('ol li'))
        items_divs = soup.find(id='zg_centerListWrapper').select('div.zg_itemImmersion')

        for item_div in items_divs:
            #print len(items)+1
            if len(item_div.select('.p13n-asin'))<1:
                #print 'skipping this above number'
                continue
            item = {}
            item['rank'] = item_div.select('span.zg_rankNumber')[0].text.replace('\n','').replace(' ','').replace('.','');
            item['asin'] = json.loads(item_div.select('.p13n-asin')[0].attrs['data-p13n-asin-metadata'])['asin']
            item['img'] = item_div.select('.p13n-asin div.a-section img')[0].attrs['src']
            item['link'] = item_div.select('.p13n-asin a')[0].attrs['href']
            item['name'] = item_div.select('.p13n-asin div.a-section img')[0].attrs['alt']
            rating_ele = item_div.select('.p13n-asin * span.a-icon-alt')
            item['rating'] = rating_ele[0].text.split()[0] if len(rating_ele)>0 else None
            review_ele = item_div.select('.p13n-asin * a.a-size-small')
            item['review_count'] = review_ele[0].text if len(review_ele)>0 else None
            item['reviews_link'] = review_ele[0].attrs['href'] if len(review_ele)>0 else None
            price_ele = item_div.select('.p13n-asin * span.p13n-sc-price')
            item['price'] = price_ele[0].text.split()[-1] if len(price_ele) > 0 else None

            items.append(item)
        cur_page = cur_page + 1;
        if cur_page <= page_count:
            link = soup.find(id='zg_paginationWrapper').select('ol li')[cur_page-1].select('a')[0].attrs['href']
    return items

In [ ]:
products = []
for index, row in bsc.iterrows():
    print row['Name']
    category_products = getCategoryBestSellers(row['BestSellerLink'])
    for p in category_products:
        p['root_category'] = row['Name']
    products.extend(category_products)

In [ ]:
prod_df = pd.read_json(json.dumps(products),orient='records')

In [ ]:
prod_df.to_csv('output/bestseller_products.csv',index=False,encoding='utf-8')
prod_df.to_json('output/bestseller_products.json',orient='records')